# Package
* Builds docker containers for all microservices

In [ ]:
import NBInclude: @nbinclude
import JSON
@nbinclude("../TrainingAlphas/Alpha.ipynb");

In [ ]:
function copy(file::String, dst::String)
    mkpath(joinpath(dst, dirname(file)))
    cp(file, joinpath(dst, file))
end

function build(app::String, basedir::String)
    name = split(app, "/")[end]
    run(`docker build -q -t rsys/$name $basedir/$name`)
end;

In [ ]:
function fetch_media_lists(basedir::String)
    app = "$basedir/fetch_media_lists"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/fetch_media_lists", app)
    copy("notebooks/API/API", app)
    copy("environment/animeplanet", app)
    copy("environment/kitsu", app)
    copy("environment/mal", app)
    copy("environment/proxies", app)
    build(app, basedir)
end;

In [ ]:
function compress_media_lists(basedir::String)
    app = "$basedir/compress_media_lists"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/compress_media_lists", app)
    copy("notebooks/ImportDatasets", app)
    copy("notebooks/ProcessData", app)
    copy("data/processed_data", app)
    build(app, basedir)
end;

In [ ]:
function bagofwords_jl(basedir::String)
    app = "$basedir/bagofwords_jl"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/bagofwords_jl", app)
    copy("notebooks/TrainingAlphas/AlphaBase.ipynb", "$app/App/src")
    copy("notebooks/TrainingAlphas/Baseline/BaselineHelper.ipynb", "$app/App/src")
    for x in ALL_MEDIUMS
        copy("data/alphas/$x/Baseline/rating/params.jld2", "$app/App/src")
    end
    copy("data/processed_data", "$app/App/src")
    build(app, basedir)
end;

In [ ]:
function bagofwords_py(basedir::String)
    for medium in ALL_MEDIUMS
        for metric in ALL_METRICS
            app = "$basedir/bagofwords_py_$(medium)_$(metric)"
            if ispath(app)
                rm(app; recursive = true)
            end
            cp("notebooks/Microservices/bagofwords_py", app)
            copy("notebooks/TrainingAlphas/BagOfWords/bagofwords.py", app)
            copy("data/alphas/$(medium)/BagOfWords/v1/$(metric)/model.pt", app)
            copy("data/alphas/$(medium)/BagOfWords/v1/$(metric)/config.json", app)
            build(app, basedir)
        end
    end
end;

In [ ]:
function nondirectional(basedir::String)
    app = "$basedir/nondirectional"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/nondirectional", app)
    copy("notebooks/TrainingAlphas/AlphaBase.ipynb", "$app/App/src")
    copy("data/processed_data", "$app/App/src")
    for m in ALL_MEDIUMS
        copy("environment/alphas/$m/Nondirectional", "$app/App/src")
    end
    build(app, basedir)
end;

In [ ]:
function transformer_jl(basedir::String)
    app = "$basedir/transformer_jl"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/transformer_jl", app)
    copy("notebooks/TrainingAlphas/AlphaBase.ipynb", "$app/App/src")
    copy("notebooks/TrainingAlphas/Transformer/Data.ipynb", "$app/App/src")
    copy("data/alphas/all/Transformer/v1/0/config.json", "$app/App/src")
    copy("data/processed_data", "$app/App/src")
    build(app, basedir)
end;

In [ ]:
function transformer_py(basedir::String)
    for medium in ALL_MEDIUMS
        app = "$basedir/transformer_py_$(medium)"
        if ispath(app)
            rm(app; recursive = true)
        end
        cp("notebooks/Microservices/transformer_py", app)
        copy("notebooks/TrainingAlphas/Transformer/transformer.py", app)
        copy("data/alphas/$(medium)/Transformer/v1/model.pt", app)
        copy("data/alphas/all/Transformer/v1/0/config.json", app)
        build(app, basedir)
    end
end;

In [ ]:
function ensemble(basedir::String)
    app = "$basedir/ensemble"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/ensemble", app)
    copy("notebooks/TrainingAlphas/AlphaBase.ipynb", "$app/App/src")
    copy("notebooks/TrainingAlphas/Ensemble/EnsembleInputs.ipynb", "$app/App/src")
    copy("data/processed_data", "$app/App/src")
    for m in ALL_MEDIUMS
        copy("environment/alphas/$m/Linear", "$app/App/src")
    end
    build(app, basedir)
end;

In [ ]:
function get_endpoints()
    apps = [
        [
            "index",
            "fetch_media_lists",
            "compress_media_lists",
            "nondirectional",
            "bagofwords_jl",
            "transformer_jl",
            "ensemble",
        ]
        ["bagofwords_py_$(x)_$(y)" for x in ALL_MEDIUMS for y in ALL_METRICS]
        ["transformer_py_$(x)" for x in ALL_MEDIUMS]
    ]
    settings = get_settings()
    if get(settings, "cloud_hosting", false)
        domain = settings["cloud_hosting_domain"]
        tag = settings["cloud_hosting_tag"]
        d = Dict()
        for app in apps
            name = replace(app, "_" => "-")
            d[app] = "https://$(name)-$(tag)-$(domain)"
        end
        return d
    else
        return Dict(app => "http://$(app):8080" for app in apps)
    end
end

function index(basedir::String)
    app = "$basedir/index"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/index", app)
    open("environment/endpoints.json", "w") do f
        write(f, JSON.json(get_endpoints()))
    end
    copy("environment/endpoints.json", "$app/App/src")
    copy("notebooks/TrainingAlphas/AlphaBase.ipynb", "$app/App/src")
    build(app, basedir)
end;

In [ ]:
function package(basedir::String)
    apps = [
        fetch_media_lists,
        compress_media_lists,
        bagofwords_jl,
        bagofwords_py,
        nondirectional,
        transformer_py,
        transformer_jl,
        ensemble,
        index,
    ]
    Threads.@threads for app in apps
        app(basedir)
    end
end;

In [ ]:
cd(joinpath(@__DIR__, "../.."))
basedir = "notebooks/Microservices/Images";

In [ ]:
if ispath(basedir)
    rm(basedir; recursive = true)
end
mkpath(basedir)
run(`docker system prune -f`);

In [ ]:
package(basedir);